# Build an agent that converts natural language queries into SQL, executes them, and returns formatted results.


# Import Required Libraries & Get API Key

In [30]:
import os            # For accessing environment variables like API keys
import sqlite3       # For creating and interacting with an SQLite database
import requests      # To make HTTP requests to the OpenAI API
import json          # To format and parse request/response data

# Get OpenAI API key stored securely in Colab (Runtime > Secrets > OPENAI_API_KEY). Note, I have used Colab to run this code.
OPENAI_API_KEY = os.environ.get("OPENAI_API_KEY")

# Set Up the Sample Student Database

## In real life scenario, you will need to integrate the code (via API) to an existing database

In [31]:
# This function creates a local SQLite database named 'school.db' and a table called 'students'
def setup_database():
    conn = sqlite3.connect('school.db')       # Connect to or create the SQLite database file
    cursor = conn.cursor()                    # Create a cursor object to execute SQL commands

    # Create the students table if it doesn't already exist
    cursor.execute('''
        CREATE TABLE IF NOT EXISTS students (
            roll_number INTEGER PRIMARY KEY,
            name TEXT,
            department TEXT,
            grade REAL,
            attendance_percent REAL,
            graduation_year INTEGER
        )
    ''')

    # Add 5 sample student records
    sample_data = [
        (1, 'Alice Johnson', 'Science', 88.5, 92.0, 2026),
        (2, 'Bob Lee', 'Arts', 76.0, 85.5, 2025),
        (3, 'Carlos Patel', 'Engineering', 91.2, 98.0, 2026),
        (4, 'Diana Rose', 'Science', 67.8, 73.2, 2024),
        (5, 'Eva Green', 'Engineering', 82.0, 89.9, 2025)
    ]

    # Insert data into the students table (IGNORE skips if already present)
    cursor.executemany("INSERT OR IGNORE INTO students VALUES (?, ?, ?, ?, ?, ?)", sample_data)

    conn.commit()     # Save the changes to the database
    conn.close()      # Close the connection

# Run the function once to initialize the database
setup_database()
print("✅ Student database created.")


✅ Student database created.


# Define the Database Schema for the LLM

In [32]:
# This function returns a description of the 'students' table for GPT-4 to understand
def get_schema():
    return """
    Table: students
    Columns:
    - roll_number (INTEGER PRIMARY KEY)
    - name (TEXT)
    - department (TEXT)
    - grade (REAL)
    - attendance_percent (REAL)
    - graduation_year (INTEGER)
    """


# Call LLM GPT-4 API to Generate SQL

In [33]:
# This function sends a prompt to GPT-4 and gets back an SQL query
def call_openai_gpt4(prompt):
    url = "https://api.openai.com/v1/chat/completions"  # OpenAI endpoint for chat models

    # Set the HTTP headers including authorization
    headers = {
        "Authorization": f"Bearer {OPENAI_API_KEY}",      # API key for access
        "Content-Type": "application/json"                # Body will be JSON format
    }

    # Create the full prompt using schema + question
    payload = {
        "model": "gpt-4",                                # GPT-4 model for better results
        "messages": [
            {
                "role": "system",
                "content": f"You are a helpful SQL assistant. Use this schema:\n{get_schema()}\nReturn only the SQL query with no explanations."
            },
            {
                "role": "user",
                "content": prompt                         # e.g., "Generate SQL for: Who is graduating in 2025?"
            }
        ],
        "temperature": 0                                  # More deterministic output (less randomness)
    }

    # Send request to OpenAI API
    response = requests.post(url, headers=headers, data=json.dumps(payload))

    # If request was successful, extract the SQL query from the response
    if response.status_code == 200:
        result = response.json()
        return result['choices'][0]['message']['content'].strip()
    else:
        # Raise error if the request failed
        raise Exception(f"OpenAI API error: {response.status_code} - {response.text}")


# Implementing Query Execution

Breakdown of Functions

1) validate_sql(sql)
Blocks dangerous operations like DROP, DELETE, UPDATE, INSERT

2) execute_query(sql)
Connects to your local SQLite student.db

Executes the (validated) SQL query

Returns results as a list of rows

3) format_results(results)
Handles:

Errors or no results

Single-column results as plain list

Multi-column results formatted nicely with spacing

Smart formatting for currency (salary, budget)

In [34]:
# Block potentially dangerous SQL commands like DELETE, DROP, etc.
def validate_sql(sql):
    sql_lower = sql.lower()  # Convert SQL to lowercase for easier checking
    if any(cmd in sql_lower for cmd in ['drop', 'delete', 'update', 'insert']):
        raise ValueError("❌ Unsafe query blocked. Only SELECT statements are allowed.")
    return sql

In [35]:
# This function executes a safe SQL query and returns the results
def execute_query(sql):
    sql = validate_sql(sql)              # Check for safety
    conn = sqlite3.connect('school.db')  # Connect to the database
    try:
        cursor = conn.cursor()
        cursor.execute(sql)              # Run the SQL command
        return cursor.fetchall()         # Return all rows of the result
    except Exception as e:
        return f"Error: {str(e)}"        # Handle and return any error message
    finally:
        conn.close()                     # Always close the connection


In [36]:
# This function formats the database results into a clean table-like string
def format_results(results):
    if isinstance(results, str):
        return results                   # If it's an error message, return as is
    if not results:
        return "No results found."       # Handle empty query results

    lines = []
    for row in results:
        # Convert each row into a string, formatting numbers to 2 decimal places
        lines.append("\t".join(str(item) if not isinstance(item, float) else f"{item:.2f}" for item in row))
    return "\n".join(lines)              # Join all rows with newlines


# Full Agent: Ask, Convert, Execute, Format

In [37]:
# This is the final agent function that connects everything together
def query_agent(question):
    try:
        # Convert natural language to SQL
        sql = call_openai_gpt4(f"Generate SQL for: {question}")
        print(f"🧠 Generated SQL:\n{sql}\n")

        # Run the SQL query and return formatted results
        results = execute_query(sql)
        return format_results(results)
    except Exception as e:
        return f"Error: {str(e)}"


# Test the Agent with Example Questions

In [38]:
# These are example questions students might ask the AI agent
test_questions = [
    "List all students",
    "Show students with grade above 85",
    "Who is graduating in 2025?",
    "What is the average attendance for each department?",
    "DROP TABLE students"  # ⚠️ This should be blocked by validate_sql()
]

# Loop through questions and show results
for question in test_questions:
    print(f"\n❓ Question: {question}")
    print(f"✅ Answer:\n{query_agent(question)}")



❓ Question: List all students
🧠 Generated SQL:
SELECT * FROM students;

✅ Answer:
1	Alice Johnson	Science	88.50	92.00	2026
2	Bob Lee	Arts	76.00	85.50	2025
3	Carlos Patel	Engineering	91.20	98.00	2026
4	Diana Rose	Science	67.80	73.20	2024
5	Eva Green	Engineering	82.00	89.90	2025

❓ Question: Show students with grade above 85
🧠 Generated SQL:
SELECT * FROM students WHERE grade > 85;

✅ Answer:
1	Alice Johnson	Science	88.50	92.00	2026
3	Carlos Patel	Engineering	91.20	98.00	2026

❓ Question: Who is graduating in 2025?
🧠 Generated SQL:
SELECT * FROM students WHERE graduation_year = 2025;

✅ Answer:
2	Bob Lee	Arts	76.00	85.50	2025
5	Eva Green	Engineering	82.00	89.90	2025

❓ Question: What is the average attendance for each department?
🧠 Generated SQL:
SELECT department, AVG(attendance_percent) 
FROM students 
GROUP BY department;

✅ Answer:
Arts	85.50
Engineering	93.95
Science	82.60

❓ Question: DROP TABLE students
🧠 Generated SQL:
DROP TABLE students;

✅ Answer:
Error: ❌ Unsafe query blocke